In [ ]:
import pickle
import numpy as np
import math
import itertools
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
type_to_ix = {'person': 0, 'user': 1, 'song': 2}
relation_to_ix = {'song_person': 0, 'person_song': 1, 'user_song': 
                  2, 'song_user': 3, '#UNK_RELATION': 4, '#END_RELATION': 5, '#PAD_TOKEN': 6}

Construct mappings from entity, type, and relation to idxs

In [ ]:
#entity vocab set is combination of songs, users, and persons
with open('../song_data/song_id.txt', 'rb') as handle:
    songs = pickle.load(handle)
    
with open('../song_data/user_id.txt', 'rb') as handle:
    users = pickle.load(handle)
    
with open('../song_data/person_song.dict', 'rb') as handle:
    person_song = pickle.load(handle)
    persons = list(person_song.keys())
    
entities = np.concatenate((songs, users, persons), axis=None)

#if we have singe map for all entities need to ensure no duplicates across categories
assert len(entities) == len(songs) + len(persons) + len(users)
assert len(entities) == len(set(entities))

entity_to_ix = {entity: idx for idx, entity in enumerate(set(entities))}
    

Construct inverse idx mappings

In [ ]:
ix_to_type = {v: k for k, v in type_to_ix.items()}
ix_to_relation = {v: k for k, v in relation_to_ix.items()}
ix_to_entity = {v: k for k, v in entity_to_ix.items()}

Save idx mappings as .dict files

In [ ]:
with open('type_to_ix.dict', 'wb') as handle:
    pickle.dump(type_to_ix, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('relation_to_ix.dict', 'wb') as handle:
    pickle.dump(relation_to_ix, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('entity_to_ix.dict', 'wb') as handle:
    pickle.dump(entity_to_ix, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('ix_to_type.dict', 'wb') as handle:
    pickle.dump(ix_to_type, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('ix_to_relation.dict', 'wb') as handle:
    pickle.dump(ix_to_relation, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('ix_to_entity.dict', 'wb') as handle:
    pickle.dump(ix_to_entity, handle, protocol=pickle.HIGHEST_PROTOCOL)

Output preview

In [ ]:
with open('type_to_ix.dict', 'rb') as handle:
    print(pickle.load(handle))
    
with open('relation_to_ix.dict', 'rb') as handle:
    print(pickle.load(handle))
    
with open('entity_to_ix.dict', 'rb') as handle:
    print(pickle.load(handle))